In [1]:
import pandas as pd
from collections import OrderedDict

In [2]:
# 5.1 Identifying Trusted Authorities

In [3]:
def get_cert_info(filename):
    unique_certs = set()
    
    with open(filename) as f:
        for line in f:
            orgs = map(str.strip, line.split('|'))

            # start at 1 to skip ultimate subject
            for i in xrange(1, len(orgs)-1):
                intermediate = orgs[i]
                issuer = orgs[i+1]
                
                unique_certs.add(intermediate + " | " + issuer)
    
    # how many intermediate certificates
    print(str(len(unique_certs)) + " certs")
    print
    
    # who controls them
    orgs = set()
    countries = set()
    
    for cert in unique_certs:
        controller = cert.split('|')[0].split(',')
        
        org = controller[-1].strip()
        orgs.add(org)
        
        country = controller[0].strip()
        if (len(country) == 2):
            countries.add(country)
            
    print(str(len(orgs)) + " orgs")
    for org in orgs:
        print org
    print
    
    print(str(len(countries)) + " countries")
    for country in countries:
        print country

In [4]:
get_cert_info('output/first_party_certs.txt')

5 certs

5 orgs
InCommon RSA Server CA
RapidSSL RSA CA 2018
GeoTrust RSA CA 2018
Let's Encrypt Authority X3
USERTrust RSA Certification Authority

1 countries
US


In [5]:
get_cert_info('output/third_party_certs.txt')

35 certs

32 orgs
COMODO RSA Certification Authority
COMODO RSA Organization Validation Secure Serv
COMODO ECC Certification Authority
DigiCert SHA2 Extended Validation Server CA
GlobalSign CloudSSL CA - SHA256 - G3
Go Daddy Class 2 Certification Aut
AlphaSSL CA - SHA256 - G2
ca@trustwave.com
Let's Encrypt Authority X3
DigiCert Baltimore CA-2 G2
DigiCert Global Root G2
Go Daddy Root Certificate Authority - G2
GlobalSign
DigiCert SHA2 Secure Server CA
Amazon
COMODO ECC Domain Validation Secure Server CA
DigiCert Global CA G2
COMODO RSA Domain Validation Secure Server CA
Thawte RSA CA 2018
DigiCert ECC Secure Server CA
Google Internet Authority G3
GeoTrust RSA CA 2018
Go Daddy Secure Certificate Authority - G2
RapidSSL RSA CA 2018
Starfield Services Root Certificate Authority
GlobalSign Domain Validation CA - SHA256 - G2
Starfield Secure Certificate Authority - G2
CloudFlare Inc ECC CA-2
GlobalSign Root CA
Amazon Root CA 1
GlobalSign Organization Validation CA - SHA256
DigiCert SHA2 High

In [6]:
# 5.2 Sources of Intermediates

In [7]:
def check_result_dict(results):
    desc_results = OrderedDict(sorted(results.items(), key=lambda t: t[1], reverse=True))
    
    for key, value_set in desc_results.iteritems():
        print(key + ": " + str(len(value_set)))

In [8]:
def num_intermediates_per_root(filename):
    authorized_pairs = dict()
    root_ca = set()

    with open(filename) as f:
        for line in f:
            orgs = map(str.strip, line.split('|'))
            root_ca.add(orgs[-1])

            # start at 1 to skip ultimate subject
            for i in xrange(1, len(orgs)-1):
                intermediate = orgs[i]
                issuer = orgs[i+1]

                # add intermediate to issuer authorized set
                if (issuer in authorized_pairs):
                    auth = authorized_pairs[issuer]
                else:
                    auth = set()

                auth.add(intermediate)
                authorized_pairs[issuer] = auth
                
    # aggregate total number approved by root
    aggregated = dict()
    for issuer, intermediates in authorized_pairs.iteritems():
        if (issuer in root_ca):
            # direct intermediates
            aggregated[issuer] = authorized_pairs[issuer]

            # indirect intermediates
            for inter in intermediates:
                if (inter in authorized_pairs):

                    auth = aggregated[issuer] | authorized_pairs[inter]
                    aggregated[issuer] = auth

    check_result_dict(aggregated)

In [9]:
num_intermediates_per_root('output/first_party_certs.txt')

US, DigiCert Inc, www.digicert.com, DigiCert Global Root CA: 2
SE, AddTrust AB, AddTrust External TTP Network, AddTrust External CA Root: 2
Digital Signature Trust Co., DST Root CA X3: 1


In [10]:
num_intermediates_per_root('output/third_party_certs.txt')

BE, GlobalSign nv-sa, GlobalSign CloudSSL CA - SHA256 - G3: 5
US, DigiCert Inc, www.digicert.com, DigiCert Global Root CA: 5
SE, AddTrust AB, AddTrust External TTP Network, AddTrust External CA Root: 5
IE, Baltimore, CyberTrust, Baltimore CyberTrust Root: 2
US, Starfield Technologies, Inc., Starfield Class 2 Certification Au: 2
BE, GlobalSign nv-sa, GlobalSign Organization Validation CA - SHA256: 5
Digital Signature Trust Co., DST Root CA X3: 1
GlobalSign Root CA - R2, GlobalSign, GlobalSign: 1
BE, GlobalSign nv-sa, Root CA, GlobalSign Root CA: 6
US, Arizona, Scottsdale, GoDaddy.com, Inc., http://certs.godaddy.com/repositor, Go Daddy Secure Certificate Authority - G2: 2
US, The Go Daddy Group, Inc., Go Daddy Class 2 Certification Aut: 2
US, SecureTrust Corporation, SecureTrust CA: 1
BE, GlobalSign nv-sa, GlobalSign Domain Validation CA - SHA256 - G2: 6
US, DigiCert Inc, www.digicert.com, DigiCert High Assurance EV Root CA: 2
US, Arizona, Scottsdale, Starfield Technologies, Inc., Starfi

In [11]:
# 5.3 Distribution of Trust

In [12]:
def num_leafs_per_root(filename):
    roots_to_leafs = dict()

    with open(filename) as f:
        for line in f:
            orgs = map(str.strip, line.split('|'))
            leaf = orgs[0].split(',')[-1].strip()
            root = orgs[-1].split(',')[-1].strip()
            
            if (root in roots_to_leafs):
                leafs = roots_to_leafs[root]
            else:
                leafs = set()

            leafs.add(leaf)
            roots_to_leafs[root] = leafs
            
    for root, leafs in roots_to_leafs.iteritems():
        print(root + ": " + str(len(leafs)))

In [13]:
num_leafs_per_root('output/first_party_certs.txt')

DigiCert Global Root CA: 2
DST Root CA X3: 27
AddTrust External CA Root: 3


In [14]:
num_leafs_per_root('output/third_party_certs.txt')

Go Daddy Class 2 Certification Aut: 17
DigiCert High Assurance EV Root CA: 28
Baltimore CyberTrust Root: 5
GlobalSign CloudSSL CA - SHA256 - G3: 17
DST Root CA X3: 38
Starfield Root Certificate Authority - G2: 3
GlobalSign Root CA: 12
SecureTrust CA: 1
GlobalSign: 18
AddTrust External CA Root: 35
Starfield Class 2 Certification Au: 17
Go Daddy Secure Certificate Authority - G2: 5
DigiCert Global Root CA: 82
GlobalSign Domain Validation CA - SHA256 - G2: 2
GlobalSign Organization Validation CA - SHA256: 1
VeriSign Class 3 Public Primary Certification: 1


In [15]:
# 5.3 Distribution of Trust

In [16]:
def get_cert_counts(filename):
    cert_counts = dict()
    
    with open(filename) as f:
        for line in f:
            orgs = map(str.strip, line.split('|'))

            # start at 1 to skip ultimate subject
            for i in xrange(1, len(orgs)-1):
                intermediate = orgs[i]
                issuer = orgs[i+1]
                cert = intermediate + " | " + issuer
                
                if (cert in cert_counts):
                    freq = cert_counts[cert] + 1
                else:
                    freq = 1
                
                cert_counts[cert] = freq
    
    desc_cert_counts = OrderedDict(sorted(cert_counts.items(), key=lambda t: t[1], reverse=True))
    
    for cert, count in desc_cert_counts.iteritems():
        print(cert + ": " + str(count))

In [17]:
get_cert_counts('output/first_party_certs.txt')

US, New Jersey, Jersey City, The USERTRUST Network, USERTrust RSA Certification Authority | SE, AddTrust AB, AddTrust External TTP Network, AddTrust External CA Root: 257
US, MI, Ann Arbor, Internet2, InCommon, InCommon RSA Server CA | US, New Jersey, Jersey City, The USERTRUST Network, USERTrust RSA Certification Authority: 257
US, Let's Encrypt, Let's Encrypt Authority X3 | Digital Signature Trust Co., DST Root CA X3: 30
US, DigiCert Inc, www.digicert.com, GeoTrust RSA CA 2018 | US, DigiCert Inc, www.digicert.com, DigiCert Global Root CA: 1
US, DigiCert Inc, www.digicert.com, RapidSSL RSA CA 2018 | US, DigiCert Inc, www.digicert.com, DigiCert Global Root CA: 1


In [18]:
get_cert_counts('output/third_party_certs.txt')

US, Google Trust Services, Google Internet Authority G3 | GlobalSign Root CA - R2, GlobalSign, GlobalSign: 578
US, DigiCert Inc, www.digicert.com, DigiCert SHA2 High Assurance Server CA | US, DigiCert Inc, www.digicert.com, DigiCert High Assurance EV Root CA: 376
US, DigiCert Inc, DigiCert SHA2 Secure Server CA | US, DigiCert Inc, www.digicert.com, DigiCert Global Root CA: 291
US, Let's Encrypt, Let's Encrypt Authority X3 | Digital Signature Trust Co., DST Root CA X3: 287
US, Arizona, Scottsdale, GoDaddy.com, Inc., http://certs.godaddy.com/repositor, Go Daddy Secure Certificate Authority - G2 | US, Arizona, Scottsdale, GoDaddy.com, Inc., Go Daddy Root Certificate Authority - G2: 157
US, Arizona, Scottsdale, GoDaddy.com, Inc., Go Daddy Root Certificate Authority - G2 | US, The Go Daddy Group, Inc., Go Daddy Class 2 Certification Aut: 157
BE, GlobalSign nv-sa, GlobalSign CloudSSL CA - SHA256 - G3 | BE, GlobalSign nv-sa, Root CA, GlobalSign Root CA: 156
BE, GlobalSign nv-sa, Root CA, Glob